# Mappable Data 

Enter an address and at the end, it will save a CSV as final_data.csv in the Data folder that can be used in a Tableau workspace.

In [1]:
import pandas as pd
import tweepy
import jsonpickle
import json
import datetime
import config
import datetime
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
import time
from pandas.io.json import json_normalize
import regex as re
import pickle
import json, requests
import numpy as np
from nltk.stem import WordNetLemmatizer
import warnings; warnings.simplefilter('ignore')


## Address:

In [2]:
address = '1901, 1825 Eastshore Hwy, CA'

## Twitter API Verification:

In [3]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
user = api.me()
print (user.name)

Soam Desai


## Address Search

The below code searches for your address on mapdevelopers.com and returns both the geolocation coordinates and the upper left and lower right corners of a bounding box.

In [5]:
#run selenium webdriver to search for address and return latitude and longitude (turn this into a function)
def find_address(address):
    #establish driver and URL
    driver = webdriver.Chrome(executable_path='C:\\bin\\chromedriver')
    url = 'https://www.mapdevelopers.com/geocode_tool.php'
    driver.get(url)
    time.sleep(10)
    #fill in address and click button
    address_field = driver.find_element_by_id("address")
    address_field.send_keys(address)
    search_button = driver.find_element_by_xpath('//*[@id="search-form"]/div[1]/span[2]/button')
    search_button.click()
    #set up 
    html= driver.page_source
    time.sleep(10)
    html= driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    location = (soup.find('div', {'id':"display_lat"}).text, soup.find('div', {'id':"display_lng"}).text)    
    return location

In [6]:
latitude, longitude = find_address(address)

In [7]:
top_left = (str(float(latitude)+.36)+ ','+ str(float(longitude)+.36))
print(top_left)
bottom_right = (str(float(latitude)-.36) + ','+str(float(longitude)-.36))
print(bottom_right)

38.2329699,-121.9452664
37.5129699,-122.6652664


## Searching for live tweets

The below code pulls in tweets from the past 24 hours, and returns only tweets with geolocation data attached in a dataframe. 

In [21]:
year = datetime.datetime.now().year
month = datetime.datetime.now().month
day = datetime.datetime.now().day

In [22]:
def get_tweets(search_terms = ""):
    tweet_id = []
    tweet_time = []
    tweet_text = []
    tweet_coordinates = []
    tweet_place = []
    tweet_count = 0
    tweet_dataframe = pd.DataFrame()
    for tweet in tweepy.Cursor(api.search,
                               q=search_terms,
                               geocode =f'{latitude},{longitude},25mi',
                               since = f'{year}-{month}-{day-1}',
                               rpp=100,
                               result_type="recent",
                               include_entities=True,).items(1000):
        if tweet.coordinates != None or tweet.place !=None: 
            tweet_id.append(tweet.id_str)
            tweet_time.append(tweet.created_at)
            tweet_text.append(tweet.text)
            tweet_coordinates.append(tweet.coordinates)
            tweet_place.append(tweet.place)
            tweet_count += 1
        else:
            pass
    tweet_dataframe['id'] = tweet_id
    tweet_dataframe['time'] = tweet_time
    tweet_dataframe['text'] = tweet_text
    tweet_dataframe['coordinates'] =tweet_coordinates
    tweet_dataframe['place'] = tweet_place
    
    print(tweet_count)
    return tweet_dataframe

In [23]:
search_terms = ['road closed', "road closure", 'intersection blocked', 
                'blocked road', 'flooded road', "closed due to road",
               'intersection closed', 'highway closed', 'hwy closed',
               'street closed', 'street flooded']

In [24]:
raw_dataframe = pd.DataFrame(columns = ['id', 'time', 'text', 'coordinates', 'place'])

In [25]:
for term in search_terms:
    df = get_tweets(term)
    raw_dataframe = raw_dataframe.append(df)
    
raw_dataframe.head()

0
0
1
0
0
0
0
0
0
0
0


,id,time,text,coordinates,place
0,1359601049930240005,2021-02-10 20:32:05,Intersection of Essex St &amp; Folsom St\n\nBl...,"{'type': 'Point', 'coordinates': [-122.3955847...",Place(_api=<tweepy.api.API object at 0x0000023...


In [26]:
#Clean data and run through model
data = raw_dataframe.drop_duplicates(subset='text')
data.shape

(1, 5)

In [27]:
#Clean data according to final modeling notbook
data = data.drop_duplicates(subset='text')
data['text'] = [i.lower() for i in data['text']]
data['text'] = [re.sub(r'@[A-z0-9]*', r' ', i) 
                    for i in data['text']];
data['text'] = [re.sub('[^A-Za-z0-9#]+', ' ', i) 
                    for i in data['text']];
lm = WordNetLemmatizer()
data['text'] = [" ".join([lm.lemmatize(w) for w in i.split()]) 
                   for i in data['text']];

In [28]:
#Import final model to read tweets using Pickle:
rating_model = pickle.load(open('C:\\Users\\desai\\Desktop\\roads\\Project_Notebooks\\gb_model.sav', 'rb'))
tvec = pickle.load(open('C:\\Users\\desai\\Desktop\\roads\\Project_Notebooks\\tvec.sav', 'rb'))

In [29]:
transformed_text = tvec.transform(data['text'])

In [30]:
data['ratings'] = rating_model.predict(transformed_text)

In [31]:
data['ratings']

0    0
Name: ratings, dtype: int64

### Filter out any lane closures:

In [32]:
word_list1 = ['road', 'street','rd', 'hwy', 'highway', 'ave', 'avenue','intersection']
word_list2 = ['closed','closure', 'blocked', 'flooded']
not_word_list = ['lane closed', 'lane closure','cleared', 're opened', 'reopen']

In [33]:
data['is_road_closure'] = data['text'].map(lambda x: 1 if ((any(word in x for word in word_list1))
                                                           & (any(word in x for word in word_list2))
                                                           & (not any(word in x for word in not_word_list))
                                                          ) else 0)

### Create dataframe for next steps

In [34]:
attempt_to_map = data[(data['ratings']==1) & (data['is_road_closure'] == 1)]

## Append Easily mappable tweets to dataframe

In [35]:
# Drop all columns without exact coordinates
map_first = attempt_to_map.dropna()

Pull out longitude and latitude, and modify data to be in final dataframe format:

In [36]:
map_first[ 'LONGITUDE'] = [point['coordinates'][0] for point in map_first['coordinates']]
map_first['LATITUDE'] = [point['coordinates'][1] for point in map_first['coordinates']]
map_first['start_end'] = ['Twitter' for i in map_first['ratings']]
map_first['incident'] = map_first['time'] 

In [37]:
final_mapping_df = map_first[['LATITUDE', 'LONGITUDE', 'start_end', 'incident']]
final_mapping_df.to_csv('C:\\Users\\desai\\Desktop\\roads\\data\\final_data5.csv')

## Parse remaining tweets for mappable intersections

As explained in our project documentation, we did not execute code parsing the tweets, as we were unable to generalize it to read tweets written by normal twitter users. For future use, the other tweets can be found in the below dataframe:

In [38]:
map_second = attempt_to_map[attempt_to_map['coordinates'].isna()]
map_second.reset_index(inplace=True)

## Pull HERE.com data

In [39]:
#set app_id and code
YOUR_APP_ID= ''
YOUR_APP_CODE= ''

In [40]:
url = f'https://traffic.ls.hereapi.com/traffic/6.3/incidents.json?bbox={top_left};{bottom_right}&apiKey=ENTER HERE&criticality=critical'
print(url)

https://traffic.ls.hereapi.com/traffic/6.3/incidents.json?bbox=38.2329699,-121.9452664;37.5129699,-122.6652664&apiKey=v5C8Pz4ujlMFgUtqGucUkumZmB_pqkCJ3R3IKmIjl8M&criticality=critical


In [41]:
res = requests.get(url)
print(res)
date_requested = time.strftime('%Y-%m-%d')
time_requested = time.strftime('%H:%M') # 24-hour format
date_requested; time_requested


<Response [200]>


'17:45'

Using methods written out in the HERE.com data cleaning notebook, append the final_mapping_df with live traffic data streamed from HERE.com

In [42]:
hmaps_json = res.json()
incidents = pd.DataFrame(hmaps_json['TRAFFIC_ITEMS']['TRAFFIC_ITEM'])
incidents = incidents.drop(axis=1, columns=['ABBREVIATION', 'ORIGINAL_TRAFFIC_ITEM_ID', 'PRODUCT',
                                              'TRAFFIC_ITEM_ID', 'mid'])
incidents.rename(str.lower, axis=1, inplace=True)
locations = pd.DataFrame(list(incidents['location']))
locations.rename(str.lower, axis=1, inplace=True)
locations = locations.applymap(lambda x: {} if pd.isnull(x) else x)
loc_coord = pd.DataFrame(list(locations['geoloc']))
loc_coord.rename(str.lower, axis=1, inplace=True)
origin = loc_coord['origin'].apply(pd.Series)
origin['start_end'] = 'start'
origin['incident'] = origin.index
to = pd.DataFrame(s for k, v in loc_coord['to'].items() for s in v)
to['start_end'] = 'end'
to['incident'] = to.index
all_locations = pd.concat((origin, to), axis=0)
all_locations.shape

(568, 4)

In [43]:
final_mapping_df = final_mapping_df.append(all_locations)

## Export final data to CSV!

In [44]:
final_mapping_df.to_csv('C:\\Users\\desai\\Desktop\\roads\\data\\final_data9.csv')

## Finally, map the road closures in Tableau